In [5]:
train_dev = 'train' #dev
data_num = 128 if train_dev == 'train' else 20
import json
from pathlib import Path
train_list = []
data_train = []
for idx in range(1, data_num):
    train_path = Path(f'data/{train_dev}/dialogues_{idx:0>3}.json')
    data_train.extend(json.load(open(train_path)))
    
for i, _ in enumerate(data_train):
    train_dict={}
    train_dict['slot'] = {}
    train_dict['context']=''
    train_dict['dialogue_id'] = data_train[i]['dialogue_id']
    train_dict['services'] = data_train[i]['services']
    for turn in data_train[i]['turns']:
        train_dict['context']+=turn['utterance']+' '
        if turn['frames']:
            service = turn['frames'][0]['service']
#             train_dict['slot'][service] = []
            if 'state' in turn['frames'][0].keys():
                for slot in turn['frames'][0]['state']['slot_values']:
                    if slot:
#                         train_dict['slot'][service].append(slot)
                        train_dict['slot'][f'{service} {slot}']=turn['frames'][0]['state']['slot_values'][slot][0]
    train_list.append(train_dict)
    
import torch
schema_path = 'data/schema.json'
with open(schema_path, encoding="utf-8") as f:
    schemas = json.load(f)
schema_dict = {}
for schema in schemas:
    schema_dict[schema['service_name']]=[f"{schema['service_name']} {slot['name']}: {slot['description']}" for slot in schema['slots']]
question_list = []
context_list = []
id_list = []
label_list = []
for i, sample in enumerate(train_list):
    for service in sample['services']:
        service_list = schema_dict[service]
        label = [0]*len(service_list)
        for slot in sample['slot']:
            slot_service, slot_name = slot.split()
            if slot_service == service and (slot_name in s for s in service_list):
                key = ''
                for s in service_list:
                    if slot_name in s:
                        key = s
                        break
                label[service_list.index(key)] = 1
        label_list.extend(label)
        question_list.extend(service_list)
        
        for slot in service_list:
            context_list.append(sample['context'])
            id_list.append(sample['dialogue_id'])
 
import csv
# 開啟輸出的 CSV 檔案
with open(f'data/{train_dev}.csv', 'w', newline='') as csvfile:
  # 建立 CSV 檔寫入器
  writer = csv.writer(csvfile)

  # 寫入一列資料
  #writer.writerow(['id','service-slot','question', 'context', 'answers'])
  writer.writerow(['id','label','slot','context'])
  for i, _ in enumerate(context_list):
          writer.writerow([id_list[i],label_list[i],question_list[i],context_list[i]])
print(f'write to {train_dev}.csv: finish')

write to train.csv: finish


In [78]:
question_list[0]

'Media_2 movie_name: Name of the movie'

In [27]:
# context max_len : 632
# num of train data : 32640

write to dev.csv: finish


In [44]:
service_list

['song_name: Name of the song',
 'artist: Name of the artist the song is performed by',
 'album: Album the song belongs to',
 'genre: Genre of the song',
 'playback_device: Playback device on which the song is to be played']

In [45]:
label

[1, 0, 1, 0, 1]

In [ ]:
id_list[-1]